In [1]:
### CNN
# convolution - subsampling - convolution - subsampling - ... - fully connected
# [                             feature extraction                                   ] [   classification  ]

In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [4]:
from tensorflow.examples.tutorials.mnist import input_data

tf.set_random_seed(777)  # reproducibility

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
# Check out https://www.tensorflow.org/get_started/mnist/beginners for
# more information about the mnist dataset

# hyper parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [5]:
### Simple CNN:
# Input layer - convolution layer 1 - Pooling layer 1 - Convolutional layer 2 - Pooling layer 2 
# - Fully-connected layer 

X = tf.placeholder(tf.float32, [None, 784])
X_img = tf.reshape(X, [-1, 28, 28, 1])  # img 28x28x1 (black/white)
Y = tf.placeholder(tf.float32, [None, 10])

# L1 ImgIn shape = (?, 28, 28, 1)
W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01))  # (3x3 filter, #color, #filter)

# Conv 통과 -> (?, 28, 28, 32)
L1 = tf.nn.conv2d(X_img, W1, strides=[1, 1, 1, 1], padding='SAME')
print(L1)
L1 = tf.nn.relu(L1)
print(L1)

# Pool: (?, 14, 14, 32) --> 2번째 convolution layer의 input
L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
print(L1)

Tensor("Conv2D:0", shape=(?, 28, 28, 32), dtype=float32)
Tensor("Relu:0", shape=(?, 28, 28, 32), dtype=float32)
Tensor("MaxPool:0", shape=(?, 14, 14, 32), dtype=float32)


In [6]:
### Conv layer 2

#L2 ImgIn shape = (?, 14, 14, 32)
W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))

# Conv -> (?, 14, 14, 64)
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
print(L2)

L2 = tf.nn.relu(L2)
print(L2)

# Pool -> (?, 7, 7, 64)
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
print(L2)

L2 = tf.reshape(L2, [-1, 7 * 7 * 64])
print(L2)

Tensor("Conv2D_1:0", shape=(?, 14, 14, 64), dtype=float32)
Tensor("Relu_1:0", shape=(?, 14, 14, 64), dtype=float32)
Tensor("MaxPool_1:0", shape=(?, 7, 7, 64), dtype=float32)
Tensor("Reshape_1:0", shape=(?, 3136), dtype=float32)


In [9]:
### Fully Connected (FC, Dense) layer

W3 = tf.get_variable("W3", shape=[7 * 7 * 64, 10], 
                     initializer=tf.contrib.layers.xavier_initializer())
b = tf.Variable(tf.random_normal([10]))  # 출력과 동일한 형태
hypothesis = tf.matmul(L2, W3) + b

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hypothesis, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

ValueError: Variable W3 already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "<ipython-input-7-f1a1fe0a8415>", line 4, in <module>
    initializer=tf.contrib.layers.xavier_initializer())
  File "/Users/shchoi/Documents/SWM_project/p1004_doif/venv/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3325, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/Users/shchoi/Documents/SWM_project/p1004_doif/venv/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3248, in run_ast_nodes
    if (await self.run_code(code, result,  async_=asy)):
  File "/Users/shchoi/Documents/SWM_project/p1004_doif/venv/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3057, in run_cell_async
    interactivity=interactivity, compiler=compiler, result=result)
  File "/Users/shchoi/Documents/SWM_project/p1004_doif/venv/lib/python3.7/site-packages/IPython/core/async_helper

In [10]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

print('Learning started. It takes some time.')
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples / batch_size)
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        feed_dict = {X: batch_xs, Y: batch_ys}
        c, _, = sess.run([cost, optimizer], feed_dict=feed_dict)
        avg_cost += c / total_batch
    print('Epoch: ', '%04d' % (epoch + 1), 'cost = ', '{:.9f}'.format(avg_cost))
    
print('Learning Finished!')

Learning started. It takes some time.
Epoch:  0001 cost =  0.345632648
Epoch:  0002 cost =  0.091713064
Epoch:  0003 cost =  0.068218277
Epoch:  0004 cost =  0.056210095
Epoch:  0005 cost =  0.046829314
Epoch:  0006 cost =  0.040964084
Epoch:  0007 cost =  0.036602715
Epoch:  0008 cost =  0.032435492
Epoch:  0009 cost =  0.027644569
Epoch:  0010 cost =  0.024589272
Epoch:  0011 cost =  0.021795083
Epoch:  0012 cost =  0.019872381
Epoch:  0013 cost =  0.016492010
Epoch:  0014 cost =  0.015082700
Epoch:  0015 cost =  0.013148659
Learning Finished!


In [11]:
correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('Accuracy: ', sess.run(accuracy, feed_dict={X: mnist.test.images, Y: mnist.test.labels}))

Accuracy:  0.9881
